# Apple Stock FAQ Chatbot

### Intelligent Virtual Assistants (IVAs)
- IVAs are quickly replacing humans as the first tier of technical support and customer service
- May interact with customers through:
  * Company Websites
  * Mobile Applications
  * Social Media
  * Customer Service Centers
  * SMS
  * Embedded Device Support

### How do IVAs work?
- User Intention (intent)
    * The interpretation of a statement or question that allows one to formulate the best possible response.
    * Used within IVAs similar to a class label within a classification task.
    Example: 'Can I bring my dog with me on board?' 
    User Intention: pet policy(dogs)

### Humans are Relational
- We relate through conversation
- We want our positions validated and understood
    - Not just what I said but why I said it 
- In customer service:
    * Both parties use relational strategies to build trust
    * We want customer service representatives to take our side
        - Persuade, manipulate, appeal to emotion

### What is the Problem?
- Ex: I need a new ticket, my dog ate my last one
- The IVA won't recognize relational, and it is design to assume that all text are relevant

### Relating with IVAs
- Early experiences: ELIZA and PARRY
- Relational language examples: Justification, backstory, greetings
    "Hello, my father is elderly and takes a while getting on a plane. Do you offer boarding assitance?"
- Increasingly used by customers as IVAs become more humanlike

### Relational Language Tags
- Greetings
- Backstory
- Justification 
- Ranting
- Gratitude
- Expressing Emotion
- Other (for language that is not relational in nature)

### What to do with Relational Segments
1. We can ignore them
    - Help intent recognition, but not IVA relational ability
2. We can deal with them
    - Determine which relational strategies are in play
    - Decide how to modify the task response in light of the relational components



## Rasu
- config.yml 
    * all the machine learning settings
- domain.yml
    * overview of intents + responses custom actions
- stories.md
    * examples of intent/action sequences
- nlu.md
    * examples of intents/entities

In [1]:
# import 
import numpy as np

import nltk 
import tflearn 
import tensorflow 
from nltk.stem.lancaster import LancasterStemmer
import random
import json
import pickle
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer

Instructions for updating:
non-resource variables are not supported in the long term


For unzipping a Tar files
https://www.winzip.com/win/en/tar-file.html

In [2]:
stemmer = LancasterStemmer()

Loading the Json file as a dictionary

In [3]:
with open("intents.json") as file:
    data = json.load(file)
data['intents'][:1]

[{'tag': 'greeting',
  'patterns': ['Hi there',
   'How are you',
   'Is anyone there?',
   'Hey',
   'Hola',
   'Hello',
   'Good day'],
  'responses': ['Hello, thanks for asking',
   'Good to see you again',
   'Hi there, how can I help?'],
  'context': ['']}]

### Data Structure
* Tags: Possible classes of user intention for asking a question
* Patterns: The ways in which users usually ask questions relating to a particular tag
* Responses: Predefined responses for each tag in the dataset from which the model can choose to respond to a particular question
* Context: Contextual words relating to a tag for easy and better classification of what the user intends with their request

Getting the root of the word by tokenizing the word and stemming it 

## Data Pre-processing

In [4]:
# try: 
#     # x if you changed the .json uncommented and add a x
#     # if the data is already saved then it runs the save data
#     with open('data.pickle', 'rb') as f:
#         words, labels, training, output = pickle.load(f)
# except:
words = []
labels = []
docs_x = []
docs_y = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        ## seprating the word into a list 
        wrds = nltk.word_tokenize(pattern)
        ## adding all to the list
        words.extend(wrds)
        ## append the tokenize words
        docs_x.append(wrds)
        ## append the tag of the intent
        docs_y.append(intent['tag'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
## lowering the words to avoid confusing
## removing the ? 
new_words = [stemmer.stem(w.lower()) for w in words if w not in '?']
## removing all the ASCII
words = []
for word in new_words:
    word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    words.append(word)
## Removing all the duplicates
words = sorted(list(set(words)))

labels = sorted(labels)

## creating a bagged of words in binary to train the model 
## So we can do one hot-encoding with the words
training = []
output = []

## list of tags into one hot-encoding
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]

    for w in words: 
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)
## switching the list into an array for input into a model
training = np.array(training)
output = np.array(output)
## saving the preprocessing 
with open('data.pickle', 'wb') as f:
    pickle.dump((words, labels, training, output), f)

In [5]:
## defines the input shape for the model
net = tflearn.input_data(shape = [None, len(training[0])])
## adding to the neural network to 2 hidden layers 
## more hidden layers for more complex problem
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
## output layers activation will allow us to get probablity for each neuron.
net = tflearn.fully_connected(net, len(output[0]), activation = 'softmax')
net = tflearn.regression(net)
## Deep Neural Networl model
model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 0.05113 | time: 0.009s
| Adam | epoch: 1000 | loss: 0.05113 - acc: 1.0000 -- iter: 32/38
Training Step: 5000  | total loss: 0.05120 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.05120 - acc: 1.0000 -- iter: 38/38
--
INFO:tensorflow:C:\Users\User\Onedrive\Desktop\github\Chatbot\Different types of Chatbot\FAQ_chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return np.array(bag)

In [8]:
def chat():
    print('Hello, I am an Apple Stock FAQ Bot!! How may I help you? (type q to stop) !')
    while True:
        inp = input('You: ')
        if inp.lower() == 'q':
            break
        ## it will giveout a probablity
        results = model.predict([bag_of_words(inp,words)])[0]
        

        ## gives an index of the largest probablity, so you can display the best answer
        results_index = np.argmax(results)
        tag = labels[results_index]
        if results[results_index] > 0.7:
#             print(results)

            for tg in data['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print('Chatbot: ' + random.choice(responses))
        else:
            print("Chatbot: Sorry, Here is Customer Service number Phone:	877-360-5390 (U.S. toll-free) for better help!")

In [9]:
results = model.predict([bag_of_words('Dividend',words)])
results

array([[8.6493057e-01, 9.0631194e-02, 4.3111991e-06, 2.3010654e-04,
        1.6312009e-07, 4.4104114e-02, 6.7618370e-05, 3.1471413e-05,
        3.3963119e-07]], dtype=float32)

In [10]:
chat()

Hello, I am an Apple Stock FAQ Bot!! How may I help you? (type q to stop) !


You:  Dividend


Chatbot: Yes. Learn more about Appleâ€™s dividend history. Learn more about Appleâ€™s dividend history. https://investor.apple.com/dividend-history/default.aspx


You:  q
